# Hugging Face for Inference

Inference on an HPC system is fine for testing, however it does not make sense to host models on an HPC system, since you need to send requests off as SLURM jobs.
Still, here on the JupyterHub we can have a look at how inference works.
Ideally, we would have a GPU available, but we can also make do with CPUs.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-v0.3"  # You can use any pre-trained model here
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Set the padding token to be the EOS token (or specify a custom one if needed)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
text = "Once upon a time"
inputs = tokenizer(text, return_tensors="pt").to(device)

In [ ]:
outputs = model.generate(inputs["input_ids"], max_length=50)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

In [ ]:
batch_texts = ["Once upon a time", "The quick brown fox"]
inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
outputs = model.generate(inputs["input_ids"], max_length=50)

for i, output in enumerate(outputs):
    print(f"Input: {batch_texts[i]}")
    print(f"Generated: {tokenizer.decode(output, skip_special_tokens=True)}\n")